In [17]:
import pandas as pd
import transformers 
import numpy as np
import torch
import sys
import pickle as pkl
import scipy.stats

sys.path.insert(0, '..')
from decompose_bert import BertForMaskedLMDecomposed


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")



In [3]:
verbs = pd.read_csv("all_VERBs.csv")["WORD"]
verb_ids = []

for verb in verbs.iloc:
    i = tokenizer.encode(" " + verb, add_special_tokens=False)
    if (len(i) == 1):
        verb_ids.append(i[0])

verb_ids = torch.Tensor(verb_ids).to(int)


In [4]:
number_df = pd.read_csv("nounpp.tsv", delimiter="\t")
max_length = 16

number_df["id"] = number_df["id"].apply(lambda x: int(x[2:]))

# only keep sentences that are plural/singular or singular/plural (distractor has different number)
number_df["subject_distractor_number"] = number_df["subject_distractor_number"].apply(
    lambda x: x if x == "singular_plural" or x == "plural_singular" else np.nan
)
number_df.dropna(inplace=True)

number_df["subject_number"] = number_df["subject_distractor_number"].apply(lambda x: x.split("_")[0])
number_df["distractor_number"] = number_df["subject_distractor_number"].apply(lambda x: x.split("_")[1])

number_df["verb"] = number_df["sentence"].apply(lambda x: " " + x.split(" ")[-1])
number_df["sentence"] = number_df["sentence"].apply(lambda x: " ".join(x.split(" ")[:-1]) + "[MASK]")

number_df = number_df.drop(
    columns=["subject_distractor_number"]
    ).pivot(index=["id", "subject_number", "distractor_number", "sentence"], columns=["correctness"], values=["verb"]).reset_index()


def get_token(correct_token, wrong_token):
    # drop if longer than 1 token
    if len(correct_token) > 3 or len(wrong_token) > 3:
        return np.nan

    return [correct_token[1], wrong_token[1]]

number_df[("token", "correct")] = tokenizer(number_df[("verb", "correct")].to_list())["input_ids"]
number_df[("token", "wrong")] = tokenizer(number_df[("verb", "wrong")].to_list())["input_ids"]

number_df[("verb_tokens")] = number_df.apply(
    lambda x: get_token(x[("token", "correct")], x[("token", "wrong")]),
    axis=1)

number_df[("sentence_tokens")] = tokenizer(number_df[("sentence", "")].to_list(),
                                           max_length=max_length,
                                           padding="max_length")["input_ids"]

# drop duplicates
number_df = number_df.dropna().reset_index(drop=True).drop(columns="token")
number_df


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_2336/2445828250.py:42: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  number_df = number_df.dropna().reset_index(drop=True).drop(columns="token")


id subject_number distractor_number  \
correctness                                          
0             603       singular            plural   
1             604       singular            plural   
2             605       singular            plural   
3             606       singular            plural   
4             608       singular            plural   
..            ...            ...               ...   
556          1793         plural          singular   
557          1795         plural          singular   
558          1797         plural          singular   
559          1799         plural          singular   
560          1800         plural          singular   

                                        sentence         verb                \
correctness                                           correct         wrong   
0              The athlete behind the cats[MASK]      engages        engage   
1              The athlete behind the cats[MASK]    remembers      remember   
2            The athlete behind the chairs[MASK]     observes       observe   
3              The athlete behind the dogs[MASK]   encourages     encourage   
4            The athlete behind the trucks[MASK]       avoids         avoid   
..                                           ...          ...           ...   
556                 The women near the cat[MASK]        avoid        avoids   
557                 The women near the dog[MASK]       engage       engages   
558              The women near the window[MASK]        avoid        avoids   
559              The women near the window[MASK]       engage       engages   
560              The women near the window[MASK]   understand   understands   

                verb_tokens                                    sentence_tokens  
correctness                                                                     
0             [24255, 8526]  [101, 1996, 8258, 2369, 1996, 8870, 103, 102, ...  
1             [17749, 3342]  [101, 1996, 8258, 2369, 1996, 8870, 103, 102, ...  
2            [24451, 11949]  [101, 1996, 8258, 2369, 1996, 8397, 103, 102, ...  
3             [16171, 8627]  [101, 1996, 8258, 2369, 1996, 6077, 103, 102, ...  
4             [26777, 4468]  [101, 1996, 8258, 2369, 1996, 9322, 103, 102, ...  
..                      ...                                                ...  
556           [4468, 26777]  [101, 1996, 2308, 2379, 1996, 4937, 103, 102, ...  
557           [8526, 24255]  [101, 1996, 2308, 2379, 1996, 3899, 103, 102, ...  
558           [4468, 26777]  [101, 1996, 2308, 2379, 1996, 3332, 103, 102, ...  
559           [8526, 24255]  [101, 1996, 2308, 2379, 1996, 3332, 103, 102, ...  
560           [3305, 19821]  [101, 1996, 2308, 2379, 1996, 3332, 103, 102, ...  

[561 rows x 8 columns]

In [5]:
def make_component_masks(sentence_tokens):
    # prep = [639, 13276, 583] # beside, near, behind
    prep = tokenizer.encode(' behind beside near')[1:-1]

    # always starts with 0, 133
    i = 2
    init_i = 0
    The_i = 1
    subj_i = []

    while sentence_tokens[i] not in prep:
        subj_i.append(i)
        i += 1
        
    prep_i = i
    the_i = i + 1
    distractor_id = []
    i += 2

    while sentence_tokens[i] != tokenizer.mask_token_id:
        distractor_id.append(i)
        i += 1
    
    mask_i = i
    
    # place each one in a separate array 
    component_masks = np.zeros((6, max_length))
    for n, component in enumerate([
        init_i, The_i, subj_i, prep_i, the_i, distractor_id
    ]):
        component_masks[n, component] = 1
    
    if mask_i != 6:
        print(sentence_tokens)
        print(component_masks)
        print(mask_i)
        print()
    return component_masks, mask_i


number_df["beta_mask"] = number_df["sentence_tokens"].apply(make_component_masks)


[101, 1996, 8258, 3875, 1996, 4624, 2015, 103, 102, 0, 0, 0, 0, 0, 0, 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
7

[101, 1996, 8258, 2379, 1996, 4624, 2015, 103, 102, 0, 0, 0, 0, 0, 0, 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
7

[101, 1996, 5916, 2379, 1996, 4624, 2015, 103, 102, 0, 0, 0, 0, 0, 0, 0]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

In [6]:
def get_proportion_contribution(n, decomposed_model):
    beta_masks = torch.tensor(number_df["beta_mask"][n][0])
    verb_tokens = torch.tensor(number_df["verb_tokens"][n])
    inputs = torch.tensor(number_df["sentence_tokens"][n]).unsqueeze(0)
    mask_i = number_df["beta_mask"][n][1]

    contribution_logits = torch.zeros((6, 2))
    # print(tokenizer.convert_ids_to_tokens(verb_tokens))

    for i, mask in enumerate(beta_masks):
        beta_mask = torch.stack([mask, 1 - mask]).unsqueeze(0)

        with torch.no_grad():
            result = decomposed_model(input_ids=inputs,
                            beta_mask=beta_mask,
                            num_contributions=2)
        result = result[0, 0, mask_i, :]

        # normalize across all verbs
        result = result - result[verb_ids].mean()
        result = result / result[verb_ids].std()

        # logits = result[verb_tokens]
        # logits = result
        # beta_z_t / z_t
        # logits = decomp_activation(logits, torch.sigmoid)
        contribution = result[verb_tokens]
        contribution_logits[i, :] = contribution

    return contribution_logits


In [9]:
sps = []
pss = []

with open("multibert_results.txt", 'a') as f:
    for n in range(0, 25):
        model_name = f"MultiBertGunjanPatrick/multiberts-seed-{n}"
        if n == 11:
            # https://huggingface.co/MultiBertGunjanPatrick/multiberts-seed-11 
            # latest commit does not contain the model files for some reason
            # use older commit instead
            decomposed_model = BertForMaskedLMDecomposed.from_pretrained(
                model_name, revision="a6b3771a09d37a29953fdf28b146b8da34e29f0e",
                debug=False
            )
        else:
            decomposed_model = BertForMaskedLMDecomposed.from_pretrained(
                model_name,
                debug=False
            )
        print()
        print(model_name)

        contribution_logits_sp = []
        contribution_logits_ps = []

        for i in range(len(number_df)):
            if number_df["subject_number"][i] == "singular":
                contribution_logits_sp.append(get_proportion_contribution(i, decomposed_model))
            else:
                contribution_logits_ps.append(get_proportion_contribution(i, decomposed_model))
            if i % 20 == 0:
                print(i)

        contribution_logits_sp = torch.stack(contribution_logits_sp)
        contribution_logits_ps = torch.stack(contribution_logits_ps)

        sp = contribution_logits_sp.mean(0).numpy()
        ps = contribution_logits_ps.mean(0).numpy()
        sps.append(sp)
        pss.append(ps)

        f.write(model_name + "\n")
        f.write(np.array2string(sp) + "\n")
        f.write(np.array2string(ps) + "\n\n")
        f.flush()



BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.



MultiBertGunjanPatrick/multiberts-seed-11
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560

MultiBertGunjanPatrick/multiberts-seed-12
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560

MultiBertGunjanPatrick/multiberts-seed-13
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560

MultiBertGunjanPatrick/multiberts-seed-14
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560

MultiBertGunjanPatrick/multiberts-seed-15
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json:   0%|          | 0.00/597 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]


MultiBertGunjanPatrick/multiberts-seed-23
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560


config.json:   0%|          | 0.00/597 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]


MultiBertGunjanPatrick/multiberts-seed-24
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560


In [11]:
# pkl.dump(pss, open("pss_save.pkl", "wb"))
# pkl.dump(sps, open("sps_save.pkl", "wb"))
pss = pkl.load(open("pss_save.pkl", "rb"))
sps = pkl.load(open("sps_save.pkl", "rb"))


In [18]:
pss = np.stack(pss)
sps = np.stack(sps)

print(pss.shape)
print(sps.shape)


(25, 6, 2)
(25, 6, 2)


In [39]:
pss.mean(0)


array([[-0.13,  0.01],
       [-0.26, -0.2 ],
       [ 1.09, -1.56],
       [ 0.03, -0.19],
       [-0.43, -0.22],
       [-0.41, -0.25]], dtype=float32)

In [40]:
sps.mean(0)


array([[ 0.22, -0.22],
       [ 0.79, -0.87],
       [ 0.85, -0.15],
       [ 0.37, -0.33],
       [ 0.05, -0.46],
       [-1.26,  0.19]], dtype=float32)

In [47]:
np.set_printoptions(precision=2, suppress=True)

scipy.stats.t.interval(0.95, pss.shape[0] - 1, loc=pss.mean(0), scale=scipy.stats.sem(pss, axis=0))


(array([[-0.34, -0.15],
        [-0.43, -0.39],
        [ 0.92, -1.91],
        [-0.12, -0.38],
        [-0.6 , -0.39],
        [-0.51, -0.4 ]]),
 array([[ 0.08,  0.17],
        [-0.09, -0.01],
        [ 1.25, -1.2 ],
        [ 0.18,  0.  ],
        [-0.27, -0.05],
        [-0.31, -0.11]]))

In [48]:
scipy.stats.t.interval(0.95, sps.shape[0] - 1, loc=sps.mean(0), scale=scipy.stats.sem(sps, axis=0))


(array([[ 0.05, -0.46],
        [ 0.57, -0.98],
        [ 0.7 , -0.29],
        [ 0.18, -0.48],
        [-0.16, -0.64],
        [-1.47,  0.03]]),
 array([[ 0.38,  0.02],
        [ 1.01, -0.76],
        [ 1.  , -0.  ],
        [ 0.55, -0.18],
        [ 0.26, -0.27],
        [-1.04,  0.35]]))